# Análisis de ejemplo de spTimer

### Modelos espacio-temporales
spTimer es una librería para análisis de experimentos que tienen una estructura espacio-temporal. Antes de empezar con spTimer, se describe muy brevemente cómo se modelan eventos espacio-temporales. 

De forma sencilla, un modelo espacio-temporal es aquel en el que se han recogido datos de forma espacial (coordenadas) y a intervalos regulares (temporal). Por ejemplo, en nuestro caso de los parquímetros, suponiendo la serie agregada, tendremos datos de ocupación ligados a una coordenada geogáfica a intervalos regulares de tiempo. Entonces, la labor del modelo es más complicada que en otros tipos: debe encontrar correlaciones espaciales y correlaciones temporales. 

### spTimer

spTimer es una librería que construye modelos espacio-temporales basándose en modelos gaussianos bayesianos. Presenta tres modelos a partir de los cuales se realizan las predicciones:

* Bayesian Gaussian Process (GP)
* Bayesian Auto-Regressive Process (AR)
* Bayesian Gaussian Predictive Processes (GPP) based AR Model

Aunque son modelos complicados, es muy importante conocer cuáles son los parámetros que se van a ajustar y cuáles son sus características.

#### Características y parámetros de los modelos de spTimer

* Constan de una parte lineal, parecida a una regresión. Estas componentes se suelen llamar **'covariates'** en la literatura, y suelen denotarse por $\bf{X_{lt}}$. Cada una de esas componentes tiene asociado un parámetro, formándse así un vector de parámetros *regresivos*, $\bf{\beta}$. 
* Aparecen dos **componentes de error** diferentes: 'nugget error', que se asocia con el término de error puro($\epsilon$), y errores aleatorios espacio-temporales ($\eta$). Matemáticamente, cada una de estas componentes se modela con una varianza ($\sigma_\epsilon^2$ y $\sigma_\eta^2$).
* Hay dos parámetros más: $\phi$, que controla el ratio con el que la correlación entre dos coordenadas decae conforme aumenta la distancia entre ellas, y $\nu$, que es el orden de la función de Bessel de segunda especie que se utiliza para calcular la correlación espacial. 
* Son **bayesianos**, en el sentido de que parten de una distribución conocida (distribución *a priori* o *prior*) de los parámetros y calculan a partir de ella una distribución final (*a posteriori*, $\pi(.|z)$).

En resumen, con cada uno de los tres modelos de spTimer, lo que se trata es de buscar los coeficientes:
* $\beta = (\beta_0, \beta_1, \cdots, \beta_p)^T$
* $\sigma_\epsilon^2$
* $\sigma_\eta^2$
* $\phi$
* $\nu$

tales que la aproximación es buena. La bondad de la aproximación se mide con el **PMCC** (Predictive Model Choice Criteria), que es un dato proporcionado por las funciones de la librería.

In [8]:
# Instalación y carga de spTimer
if (!require(spTimer)){
    install.packages("spTimer")
    library(spTimer)
} 
# Verificación carga spTimer
(.packages())

[1] "spTimer"       "RevoUtils"     "stats"         "graphics"     
 [5] "grDevices"     "utils"         "datasets"      "RevoUtilsMath"
 [9] "methods"       "base"

## 1. Descripción del dataset
Para estudiar una serie espacio temporal, necesitamos, al menos, varias coordenadas geográficas (2 en este caso: latitud y longitud) y una o varias escalas temporales. Un modelo con 2 escalas temporales, por ejemplo, sería: (año, semana). 

Para realizar un estudio previo, este notebook utiliza el dataset de taxis de Nueva York (NYdata) que se encuentra dentro de la librería spTimer.

A continuación se cargan y visualizan los datos:

In [9]:
df <- NYdata
head(df)

s.index,Longitude,Latitude,Year,Month,Day,o8hrmax,cMAXTMP,WDSP,RH
1,-73.757,42.681,2006,7,1,53.88,27.85772,5.459953,2.766221
1,-73.757,42.681,2006,7,2,57.13,30.11563,8.211767,3.197750
1,-73.757,42.681,2006,7,3,72.00,30.00001,4.459581,3.225186
1,-73.757,42.681,2006,7,4,36.63,27.89656,3.692225,4.362334
1,-73.757,42.681,2006,7,5,42.63,25.65698,4.374314,3.950320
1,-73.757,42.681,2006,7,6,30.88,24.61968,4.178086,3.420533


Además de las columnas principales (geográficas y temporales), el dataset presenta otras columnas de interés para la predicción. Por orden, son:  concentración de ozono, temperatura máxima, velocidad del viento y humedad relativa. 

Como conclusión, un dataset para este tipo de problemas debe tener:
* Una o varias columnas geográficas. Normalmente se suelen 2: latitud y longitud, aunque podrían ser otras.
* Una o varias columnas temporales. Se pueden definir varios niveles dentro de esta variable, para hacer referencia a modelos estacionales:
    * Año
    * Semana
    * Día
    * Hora
    
Estos dos bloques serían los principales. A ellos, se añaden columnas que representen una característica o propiedad del fenómeno a representar, que ocurra en unas coordeandas geográficas determiandas y en un tiempo determinado. Estas columnas son las que deben agregarse en función de la granularidad que se defina, principalmente, en las columnas temporales. 

Finalmente, se hace la predicción sobre una de estas columnas agregadas que hemos añadido. 

## 2. Fitting



Para realizar el fitting, spTimer muestrea el dataframe con **muestreo de Gibbs** (muestreo utilizado para obtener muestras aleatorias de una distribución conjunta de dos o más variables aleatorias). Después, hace el fitting según los parámetros que le indiquemos. De entre todos los parámetros que hay, los más interesantes para empezar a arrancar el modelo son:

1. Cuál es la **parte regresiva** del modelo (parámetros $\beta$). Se hace a través de un elemento 'fórmula' de R. 
1. Los **datos de origen**, con todas las columnas (data=)
1. **Tipo de modelo**: GP, AR o GPP. Es importante destacar que el GPP requiere hacer operaciones adicionales con los datos. GP y AR son más directos. La diferencia entre GP y AR es que, si bien ambos parten de modelos gaussianos, AR incorpora un término de autorregresión ($\rho$), que GP no tiene. 
1. **Coordenadas geográficas**
1. **Coordenadas temporales**

Después, se puede añadir complejidad al modelo, con el resto de opciones. Por ejemplo, se pueden definir los **priors** de los parámetros.

Por ejemplo, con el dataset de NY:


In [10]:
# Coordenadas temporales
time.data <- spT.time(t.series=60,segment=1)

# Distribuciones a priori de los parámetros 
priors <- spT.priors(model="GP",inv.var.prior=Gamm(2,1),beta.prior=Norm(0,10^4))

# Valores iniciales de los parámetros del modelo
initials <- spT.initials(model="GP", sig2eps=0.01,sig2eta=0.5, beta=NULL, phi=0.001)

# Decaimiento espacial
spatial.decay <- spT.decay(distribution=Gamm(2,1), tuning=0.08)

model <- spT.Gibbs(formula=o8hrmax~cMAXTMP+WDSP+RH, 
                   data=df, model='GP', 
                   coords=~Longitude+Latitude, 
                   #time.data=time.data,
                   priors=priors,
                   initials=initials,
                   spatial.decay=spatial.decay)
summary(model)


 Output: GP models 
---------------------------------------------------------------
 Sampled: 5000 of 5000, 100.00%.
 Batch Acceptance Rate (phi): 30.25%
 Checking Parameters: 
   phi: 0.0123, sig2eps: 2.4228, sig2eta: 103.1568
   beta[1]: -17.1807   beta[2]: 2.5892   beta[3]: 1.6207   beta[4]: -3.0163
---------------------------------------------------------------
## 
# nBurn =  1000 , Iterations =  5000 . 
# Overall Acceptance Rate (phi) =  30.24 % 
## 
##
# Elapsed time: 13.07 Sec.
##

# Model: GP 
-----------------------------------------------------
Model: GP
Call: o8hrmax ~ cMAXTMP + WDSP + RH
Iterations: 5000
nBurn: 1000
Acceptance rate for phi (%): 30.24
-----------------------------------------------------
        Goodness.of.fit  Penalty     PMCC
values:        47087.77 175941.5 223029.3
-----------------------------------------------------
Computation time: 13.07  - Sec.
-----------------------------------------------------
Parameters:
                Mean   Median      SD 

In [11]:
confint(model)

,2.5%,97.5%
(Intercept),-31.222004621,-2.24523675
cMAXTMP,1.838605798,2.69272823
WDSP,0.986045747,2.24936947
RH,-2.926400343,0.67827472
sig2eps,1.139339892,4.13067090
sig2eta,89.054032877,216.26630080
phi,0.005893714,0.01656735


In [12]:
model$PMCC

,Goodness.of.fit,Penalty,PMCC
values:,47087.77,175941.5,223029.3


## 3. Predicciones

Para este apartado, voy a utilizar un código completo de ejemplo del manual de spTimer. Con él, se ilustra cómo realizar una predicción sobre una localización concreta en una hora determinada.

Lo bueno de este ejemplo es que hace una validación cruzada, con muchas métricas, que explican cómo hacer comparaciones entre modelos. 


In [13]:
data(NYdata)

s<-c(8,11,12,14,18,21,24,28)

DataFit<-spT.subset(data=NYdata, var.name=c("s.index"), s=s, reverse=TRUE)
DataFit<-subset(DataFit, with(DataFit, !(Day %in% c(30, 31) & Month == 8)))

DataValPred<-spT.subset(data=NYdata, var.name=c("s.index"), s=c(8,11,12,14,18,21,24,28))
DataValPred<-subset(DataValPred, with(DataValPred, !(Day %in% c(30, 31) & Month == 8)))

In [14]:
head(DataFit)

s.index,Longitude,Latitude,Year,Month,Day,o8hrmax,cMAXTMP,WDSP,RH
1,-73.757,42.681,2006,7,1,53.88,27.85772,5.459953,2.766221
1,-73.757,42.681,2006,7,2,57.13,30.11563,8.211767,3.197750
1,-73.757,42.681,2006,7,3,72.00,30.00001,4.459581,3.225186
1,-73.757,42.681,2006,7,4,36.63,27.89656,3.692225,4.362334
1,-73.757,42.681,2006,7,5,42.63,25.65698,4.374314,3.950320
1,-73.757,42.681,2006,7,6,30.88,24.61968,4.178086,3.420533


In [15]:
head(DataValPred)

,s.index,Longitude,Latitude,Year,Month,Day,o8hrmax,cMAXTMP,WDSP,RH
435,8,-73.859,44.393,2006,7,1,59.00,25.94042,5.945552,3.162660
436,8,-73.859,44.393,2006,7,2,59.13,29.20228,8.557723,3.218115
437,8,-73.859,44.393,2006,7,3,52.25,27.74775,4.791014,2.888200
438,8,-73.859,44.393,2006,7,4,50.25,26.66863,4.468024,4.282619
439,8,-73.859,44.393,2006,7,5,35.63,24.82223,5.413934,3.815903
440,8,-73.859,44.393,2006,7,6,25.88,23.20626,3.815445,3.584016


In [17]:
set.seed(11)
post.gp <- spT.Gibbs(formula=o8hrmax ~cMAXTMP+WDSP+RH,
                     data=DataFit, 
                     model="AR", 
                     coords=~Longitude+Latitude,
                     scale.transform="SQRT")
print(post.gp)
pred.coords<-as.matrix(unique(cbind(DataValPred[,2:3])))

set.seed(11)
pred.gp <- predict(post.gp, newdata=DataValPred, newcoords=pred.coords)
print(pred.gp)
names(pred.gp)
spT.validation(DataValPred$o8hrmax,c(pred.gp$Mean))


 Output: AR models 
---------------------------------------------------------------
 Sampled: 5000 of 5000, 100.00%.
 Batch Acceptance Rate (phi): 0.00%
 Checking Parameters: 
   phi: 0.0051, rho: 0.4099, sig2eps: 0.0134, sig2eta: 0.7287
   beta[1]: 2.2754   beta[2]: 0.0724   beta[3]: 0.0138   beta[4]: -0.0249
---------------------------------------------------------------
## 
# nBurn =  1000 , Iterations =  5000 . 
# Overall Acceptance Rate (phi) =  0 % 
## 
##
# Elapsed time: 8.35 Sec.
##

# Model: AR 
-----------------------------------------------------
Model: AR
Call: o8hrmax ~ cMAXTMP + WDSP + RH
Iterations: 5000
nBurn: 1000
Acceptance rate for phi (%): 0
-----------------------------------------------------
        Goodness.of.fit Penalty   PMCC
values:          311.15  451.48 762.63
-----------------------------------------------------
Computation time: 8.35  - Sec.

 Prediction: AR models 
#
# Tolerance Limit (unit): 2
# Location distances are alright 
#
---------------------

[1] "pred.samples"         "pred.coords"          "distance.method"     
 [4] "Distance.matrix.pred" "cov.fnc"              "predN"               
 [7] "Mean"                 "Median"               "SD"                  
[10] "Low"                  "Up"                   "computation.time"    
[13] "model"                "type"

##
 Mean Squared Error (MSE) 
 Root Mean Squared Error (RMSE) 
 Mean Absolute Error (MAE) 
 Mean Absolute Percentage Error (MAPE) 
 Bias (BIAS) 
 Relative Bias (rBIAS) 
 Relative Mean Separation (rMSEP)
##


MSE    RMSE     MAE    MAPE    BIAS   rBIAS   rMSEP 
48.8689  6.9906  5.4625 12.9156  1.2910  0.0276  0.2937